In [1]:
#%%capture
from pyFolder.GoogleSheetsAPI import *;

C:\Users\pgathondu\Testing\client_secret_file.json-sheets-v4-(['https://www.googleapis.com/auth/spreadsheets'],)
['https://www.googleapis.com/auth/spreadsheets']
sheets service created successfully


In [2]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib;
# !pip install voila;
# !pip install pandas;
# !pip install numpy;
# !pip install --upgrade tableau-api-lib
# !pip install widgetsnbextension
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [3]:
import voila;
import numpy as np;
import pandas as pd;
from pyFolder.TableauRestAPi import *;
import ipywidgets as widgets;
from ipywidgets import *;
from IPython.display import display, clear_output;
from functools import partial
import sys

In [4]:
style = {'text-align: center', 'font-size: 500px'}
pre_login_header=widgets.HTML(value='<h1><b><p style="text-align: center;\
                        font-size: 20px;background-color:red">\
                        Tableau Rest API login please!</p></b></h1>',
             layout=Layout(border='solid_red 1px red', margin='5px')
            )
post_login_header=widgets.HTML(value='<h1><b><p style="text-align: center;\
                        font-size: 20px;background-color:powderblue">\
                        Tableau Rest API logged in</p></b></h1>',
             layout=Layout(border='solid_black 1px ', margin='5px')
            )


In [5]:
# classes
class Counter:
    def __init__(self, initial=0):
      self.value = initial
    def increment(self, amount=1):
      self.value += amount
      return self.value
    def __iter__(self, sentinal=False):
      return iter(self.increment, sentinal)

class reportList:
    def __init__(self):
      self.df_bi = pd.DataFrame()
    def getBiList(self, login_counter, login_creds):
        if len(self.df_bi) == 0 and login_counter == 1:
            self.df_bi = biReportList(login_creds)

class storePassword:
    def __init__(self):
      self.passwtuple = ()
    def getLogin(self, username, password):
        self.passwtuple = (username, password)

class logInCheck:
    def __init__(self,initial=False):
      self.loggedIn = initial
      self.loginSucess = '<Response [200]>'
      self.loginFail = '<Response [401]>'
    def confirmLogin(self, response):
        self.loggedIn = True if response == self.loginSucess else False
# class gsheetTitles:
#     def __init__(self):
#         self.wsheetNames = []
#     def addTitles(self, urlWidgetValue):
#         self.

In [6]:
# functions
def loginInteract():
    with output:
        output.clear_output()
        display(passBox)

    
def on_start_button_clicked(event):
    with output:
        output.clear_output()
        loginInteract()
def on_login_button_clicked(event):
    with output:
        output.clear_output()
        pws_is_correct.confirmLogin(checkLogIn(widget_username.value,widget_password.value))
        if pws_is_correct.loggedIn:
            login_counter.increment()
            credentials.getLogin(widget_username.value,widget_password.value)
            reports_list.getBiList(login_counter.value, credentials.passwtuple)
            if len(reports_list.df_bi) > 0:
                login_counter.increment()
            pre_login_header.close()
            startBtn.close()
            passBox.close()
            display(vbox_after_login)
            createDropdowns(reports_list.df_bi)
            display(widgets.HBox(children=[tab_nest, importBtn]))
        else:
            print('Log in Failed!')

def on_export_button_clicked(event):
    with output:
        output.clear_output()
        try:
            sendExport()
            print('Sucessfully sent export')
        except Exception as e:
           print('There was an error, try again ?') 
           print('Error mesage:',str(e))         
            
            
def update2(*args):
    df_name = reports_list.df_bi[['workbook_name','name']]
    df2 = df_name.set_index('workbook_name')
    l2 = df2.filter(like=workbook_widget.value,axis=0)['name'].unique().tolist()
    l2.sort()
    worksheet_widget.options = l2
    

def update(*args):
    df1 = reports_list.df_bi.set_index('project_name')
    l1 = df1.filter(like=project_widget.value,axis=0)['workbook_name'].unique().tolist()
    l1.sort()
    workbook_widget.options = l1
    workbook_widget.observe(update2)
    
    
def createDropdowns(df_bi):
    project_widget.options = df_bi['project_name'].unique().tolist()
    display(t)
    project_widget.observe(update)
    
def sendExport():
    Url = url_widget.value
    worksheet_name = sheetTitle_widget.value
    my_workbook = workbook_widget.value
    my_worksheet = worksheet_widget.value

    dim1= tab_nest.children[0].children[0].children[0].value
    val1 = tab_nest.children[0].children[0].children[1].value
    dim2 = tab_nest.children[1].children[0].children[0].value
    val2 = tab_nest.children[1].children[0].children[1].value

    df = tSCRestAPI(credentials.passwtuple,my_workbook, my_worksheet, dim1, val1, dim2, val2)
    gsheetId = getSheetId(Url) 
    clearSheets(gsheetId,worksheet_name)
    writeDataToSheetDf(worksheet_name,gsheetId,df)


In [7]:
#define variables
login_counter = Counter()
reports_list = reportList()
credentials = storePassword()
pws_is_correct = logInCheck()

In [8]:
# widgets
project_widget = Dropdown(layout={'width': 'max-content'})
workbook_widget = Dropdown(layout={'width': 'max-content'})
worksheet_widget = Dropdown(layout={'width': 'max-content'})
t = widgets.HBox(children=[project_widget, workbook_widget,worksheet_widget])

startBtn = widgets.Button(
    description = 'Start Rest API',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)

loginBtn = widgets.Button(
    description = 'Login',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)

importBtn = widgets.Button(
    description = 'Create Export',
    tooltip = 'Send',
    style = {'description_width': 'initial'},
    layout = {'bottom': '0px'}
)


url_widget=widgets.Text(
    value='',
    placeholder='Enter Spreadsheet Link',
    description='URL Link:',
    disabled=False
)


sheetTitle_widget=widgets.Text(
    value='',
    placeholder='Enter Spreadsheet Tab Name',
    description='Importing To:',
    disabled=False
)

widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Ringo'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

widget_username=widgets.Text(
value='',
placeholder='Type something',
description='Username:',
disabled=False
)
widget_password=widgets.Password(
value='',
placeholder='Enter password',
description='Password:',
disabled=False
)

folder_name=widgets.HTML(value='<h3><b><p style="text-align: right;\
                        font-size: 10px;background-color:orange">\
                        BI Folder Name</p></b></h3>',
             layout=Layout(border='solid_red 1px red', width='50px', align_self= 'center')
            )
workbook_name=widgets.HTML(value='<h3><b><p style="text-align: right;\
                        font-size: 10px;background-color:orange">\
                        WorkBook Name</p></b></h3>',
             layout=Layout(border='solid_red 1px red', width='100px')
            )
view_name=widgets.HTML(value='<h3><b><p style="text-align: right;\
                        font-size: 10px;background-color:orange">\
                        BI View Name</p></b></h3>',
             layout=Layout(border='solid_red 1px red', margin='5px', width = '150px')
            )
bi_files = widgets.HBox(children=[folder_name, workbook_name, view_name])


a1 = widgets.Accordion(children=[widgets.HBox(children=[widgets.Text(), widgets.Text()])],)
a1.set_title(0, 'Dimension Name, Value')
a2 = widgets.Accordion(children=[widgets.HBox(children=[widgets.Text(), widgets.Text()])],)
a2.set_title(0, 'Dimension Name, Value')

tab_nest = widgets.Tab()
tab_nest.children = [a1, a2]
for i in range(len(tab_nest.children)):
    tab_nest.set_title(i, 'filter '+str(i+1))


In [9]:

output = widgets.Output()
startBtn.on_click(on_start_button_clicked)
loginBtn.on_click(on_login_button_clicked)
importBtn.on_click(on_export_button_clicked)

In [12]:
startBox = widgets.VBox([startBtn,output])
passBox = widgets.VBox([widget_username,widget_password,loginBtn])
vbox_before_login = widgets.VBox(children=[pre_login_header,startBox])
vbox_after_login = widgets.VBox(children=[post_login_header,widgets.HBox(children=[url_widget,sheetTitle_widget]),bi_files])
display(vbox_before_login)

